# Morningstar: Entraînement sur Google Colab (avec Téléchargement de Données)

Ce notebook permet d'entraîner le modèle Morningstar en utilisant les ressources GPU de Google Colab.
Il télécharge les données nécessaires directement depuis un échange spécifié.

**Étapes:**
1.  Configurer les paramètres de téléchargement des données (échange, paire, timeframe, dates).
2.  Exécuter les cellules d'installation et de configuration.
3.  Exécuter les cellules restantes pour télécharger et préparer les données, entraîner le modèle et sauvegarder/télécharger le résultat.

## 1. Paramètres de Téléchargement des Données

In [ ]:
# --- Configurez ces paramètres --- 
exchange_name = 'binance'  # Nom de l'échange (ex: 'binance', 'kraken', 'bybit')
pair = 'BTC/USDT'         # Paire de trading à télécharger
timeframe = '1h'          # Timeframe ('1m', '15m', '30m', '1h', '4h', '6h', '1d')
start_date = '2022-01-01' # Date de début (YYYY-MM-DD)
end_date = '2024-01-01'   # Date de fin (YYYY-MM-DD)
# ---------------------------------

## 2. Installation des Dépendances et Clonage du Projet

In [ ]:
# Installation des bibliothèques nécessaires
!pip install -q tensorflow pyarrow wandb pandas numpy ccxt ta asyncpraw

# Vérification GPU
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU disponible: {gpu_devices}")
    # Configuration pour éviter les erreurs OOM sur certaines cartes
    try:
        for gpu in gpu_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth activé pour les GPUs.")
    except RuntimeError as e:
        print(f"Erreur lors de l'activation de memory growth: {e}")
else:
    print("Aucun GPU détecté. L'entraînement se fera sur CPU (peut être très lent).")

In [ ]:
# Clonage du dépôt GitHub (contient le code source du modèle et des utilitaires)
!rm -rf eva001 # Supprimer le dossier s'il existe déjà pour éviter les conflits
!git clone https://github.com/Cabrel10/eva001.git
%cd eva001

# Installation des dépendances spécifiques du projet
!pip install -q .

## 3. Configuration Weights & Biases (W&B)

In [ ]:
# Connexion à W&B (utilise la clé API fournie)
!wandb login a1478933771f0389426436c0de1c39585a5a452c

## 4. Téléchargement et Préparation des Données

Cette section télécharge les données OHLCV depuis l'échange, calcule les indicateurs techniques, et prépare les séquences et labels pour l'entraînement.

In [ ]:
import pandas as pd
import numpy as np
import logging
import asyncio
import os
import sys

# Ajouter le répertoire racine du projet cloné au sys.path pour les imports
project_root = '/content/eva001'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from Morningstar.configs.morningstar_config import MorningstarConfig
from Morningstar.utils.data_manager import ExchangeDataManager
# Importer les bibliothèques nécessaires pour les indicateurs
import ta
from Morningstar.utils.custom_indicators import volume_anomality # Importer seulement la fonction nécessaire

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Téléchargement des données --- 
logger.info(f"Téléchargement des données pour {pair} sur {exchange_name} ({timeframe}, {start_date} à {end_date})...")
manager = ExchangeDataManager(exchange_name)

async def download_data():
    await manager.load_markets_async() # Charger les marchés
    df_raw = await manager.load_data(pair, timeframe, start_date, end_date)
    await manager.close()
    return df_raw

# Exécuter la fonction asynchrone
try:
    # Utiliser nest_asyncio si déjà dans un event loop (cas de Colab/Jupyter)
    import nest_asyncio
    nest_asyncio.apply()
    df_ohlcv = asyncio.run(download_data())
except RuntimeError as e:
     # Si nest_asyncio n'est pas nécessaire ou cause une erreur, essayer sans
     if 'cannot run loop while another loop is running' in str(e):
         logger.warning("Event loop déjà actif, tentative sans nest_asyncio.")
         # Cette partie est plus complexe à gérer proprement dans un notebook.
         # Pour simplifier, on pourrait lever une erreur ou essayer une autre approche.
         # Alternative simple mais bloquante (si l'event loop existe déjà):
         loop = asyncio.get_event_loop()
         df_ohlcv = loop.run_until_complete(download_data())
     else:
        raise e
except Exception as e:
    logger.error(f"Erreur lors du téléchargement des données: {e}")
    raise

if df_ohlcv.empty:
    logger.error("Aucune donnée n'a été téléchargée. Vérifiez les paramètres (paire, dates, timeframe) et la connexion.")
    raise ValueError("Échec du téléchargement des données OHLCV.")
else:
    logger.info(f"Données OHLCV téléchargées avec succès. Shape: {df_ohlcv.shape}")

# --- Calcul des Indicateurs Techniques (Spécifiques au dataset cible) --- 
logger.info("Calcul des indicateurs techniques spécifiques...")
df_indicators = df_ohlcv.copy()
try:
    # RSI
    df_indicators['rsi'] = ta.momentum.rsi(df_indicators['close'], window=14)
    # MACD
    macd = ta.trend.MACD(df_indicators['close'])
    df_indicators['macd'] = macd.macd()
    df_indicators['macd_signal'] = macd.macd_signal()
    df_indicators['macd_hist'] = macd.macd_diff()
    # Bollinger Bands
    bollinger = ta.volatility.BollingerBands(df_indicators['close'])
    df_indicators['bb_upper'] = bollinger.bollinger_hband()
    df_indicators['bb_middle'] = bollinger.bollinger_mavg()
    df_indicators['bb_lower'] = bollinger.bollinger_lband()
    # Volume MA
    df_indicators['volume_ma'] = df_indicators['volume'].rolling(20).mean()
    # Volume Anomaly (depuis custom_indicators)
    df_indicators['volume_anomaly'] = volume_anomality(df_indicators)

    logger.info(f"Indicateurs techniques spécifiques ajoutés.")
    # Afficher les dernières lignes pour vérifier
    # print(df_indicators.tail())
except Exception as e:
    logger.error(f"Erreur lors du calcul des indicateurs: {e}")
    raise

# --- Ajout des colonnes manquantes (Pair, Social/GitHub placeholders) --- 
logger.info("Ajout des colonnes Pair et placeholders Social/GitHub...")
df_indicators['pair'] = pair # Ajouter la colonne pair
social_github_cols = ['commits', 'stars', 'forks', 'issues_opened', 'issues_closed']
for col in social_github_cols:
    df_indicators[col] = 0 # Ajouter les colonnes avec 0 par défaut

# --- Préparation Finale (Nettoyage, Normalisation, Séquences, Labels) --- 
logger.info("Début de la préparation finale des données (features, NaN, scaling, séquences, labels, split)...")
config = MorningstarConfig()
data = df_indicators.copy() # Utiliser le dataframe complété

# 1. Sélection des features (Basé sur l'échantillon fourni)
available_cols = data.columns.tolist()
features = []
def add_features(column_list):
    added = [col for col in column_list if col in available_cols]
    if len(added) != len(column_list):
         missing = set(column_list) - set(available_cols)
         logger.warning(f"Colonnes de config manquantes après calcul indicateurs: {missing}")
    return added

# Définir explicitement les features basées sur l'échantillon
target_features = [
    'open', 'high', 'low', 'close', 'volume', # Base OHLCV
    'rsi', 'macd', 'macd_signal', 'macd_hist', # Indicateurs
    'bb_upper', 'bb_middle', 'bb_lower', 
    'volume_ma', 'volume_anomaly',
    'commits', 'stars', 'forks', 'issues_opened', 'issues_closed' # Placeholders Social/GitHub
]

# Vérifier si toutes les colonnes cibles existent après calculs/ajouts
missing_cols = [col for col in target_features if col not in data.columns]
if missing_cols:
    raise ValueError(f"Colonnes cibles manquantes dans le DataFrame préparé: {missing_cols}")

features = target_features # Utiliser cette liste pour la suite
logger.info(f"Colonnes finales sélectionnées pour le modèle ({len(features)}): {features}")
data = data[features].copy() # Sélectionner uniquement ces colonnes pour la suite

# 2. Gestion des NaN (après calcul indicateurs et ajout placeholders)
initial_rows = len(data)
# Remplissage spécifique (ex: 0 pour indicateurs) ou ffill/bfill
# Simple ffill pour commencer, puis dropna
data.fillna(method='ffill', inplace=True)
data.fillna(method='bfill', inplace=True) # Pour les NaN au début
data.dropna(inplace=True)
rows_dropped = initial_rows - len(data)
if rows_dropped > 0:
    logger.warning(f"{rows_dropped} lignes supprimées à cause de NaN restants après ffill/bfill.")
if data.empty:
    raise ValueError("Le DataFrame est vide après la suppression des NaN.")
logger.info(f"Données après gestion NaN: {len(data)} lignes")

# 3. Normalisation (StandardScaler par groupe)
def scale_group(df, cols):
    valid_cols = [col for col in cols if col in df.columns]
    if valid_cols:
         mean = df[valid_cols].mean()
         std = df[valid_cols].std()
         std[std == 0] = 1 # Éviter division par zéro
         df[valid_cols] = (df[valid_cols] - mean) / std
         logger.info(f"Normalisation appliquée aux colonnes: {valid_cols}")
    return df

data = scale_group(data, config.base_columns)
data = scale_group(data, config.technical_columns)
# data = scale_group(data, config.social_columns)
# data = scale_group(data, config.correlation_columns)

# 4. Création des séquences
time_window = 60 # À définir ou récupérer depuis config si elle existe
final_features = data.columns.tolist()
logger.info(f"Création des séquences (time_window={time_window}) avec {len(final_features)} features...")

total_sequences = len(data) - time_window
if total_sequences <= 0:
    raise ValueError(f"Pas assez de données ({len(data)} lignes) pour créer des séquences avec une fenêtre de {time_window}.")

# Utiliser float32 pour économiser mémoire
sequences = np.zeros((total_sequences, time_window, len(final_features)), dtype=np.float32)
data_values = data[final_features].values.astype(np.float32)

# Vectorisation pour créer les séquences (plus rapide que la boucle par chunk pour Colab)
shape = (data_values.shape[0] - time_window + 1, time_window, data_values.shape[1])
strides = (data_values.strides[0], data_values.strides[0], data_values.strides[1])
sequences = np.lib.stride_tricks.as_strided(data_values, shape=shape, strides=strides)
logger.info(f"Séquences créées. Shape: {sequences.shape}")

# 5. Génération des labels (Triple Barrier - réplication de la logique)
def _generate_labels(sequences: np.ndarray, data_df: pd.DataFrame, look_forward_steps: int = 5, pt_sl_ratio: float = 1.5) -> np.ndarray:
    logger.info(f"Génération des labels (Triple Barrier): look_forward={look_forward_steps}, pt_sl_ratio={pt_sl_ratio}")
    n_samples = sequences.shape[0]
    close_col_index = final_features.index('close') # Index de la colonne 'close'
    entry_prices = sequences[:, -1, close_col_index]
    labels = np.zeros((n_samples, 3), dtype=np.float32)
    labels[:, 2] = 1 # Initialiser à Hold

    # Utiliser l'index du DataFrame original pour les prix futurs
    future_prices = np.full((n_samples, look_forward_steps), np.nan, dtype=np.float32)
    close_values = data_df['close'].values
    # L'index de fin de la première séquence dans data_df est time_window - 1
    # L'index de fin de la i-ème séquence est (time_window - 1) + i
    for i in range(n_samples):
        end_seq_idx = (time_window - 1) + i
        start_future_idx = end_seq_idx + 1
        end_future_idx = start_future_idx + look_forward_steps
        if end_future_idx <= len(close_values):
            future_prices[i] = close_values[start_future_idx:end_future_idx]

    # Barrières simples (pourcentage fixe - à améliorer avec volatilité si besoin)
    volatility = 0.01 # Approximation simple
    upper_barrier = entry_prices * (1 + volatility * pt_sl_ratio)
    lower_barrier = entry_prices * (1 - volatility)

    for t in range(look_forward_steps):
        current_future_prices = future_prices[:, t]
        valid_indices = ~np.isnan(current_future_prices)
        hold_indices = labels[:, 2] == 1
        active_indices = valid_indices & hold_indices
        if not np.any(active_indices): break # Arrêter si plus de labels à définir

        # Indices actifs pour cette itération
        current_active_indices = np.where(active_indices)[0]

        # Vérifier barrière supérieure
        hit_upper_mask = current_future_prices[current_active_indices] >= upper_barrier[current_active_indices]
        hit_upper_indices = current_active_indices[hit_upper_mask]
        if len(hit_upper_indices) > 0:
             labels[hit_upper_indices] = [1, 0, 0] # Buy

        # Mettre à jour les indices qui sont toujours 'Hold'
        still_hold_mask = labels[current_active_indices, 2] == 1
        active_indices_after_upper = current_active_indices[still_hold_mask]

        # Vérifier barrière inférieure pour ceux qui sont toujours 'Hold'
        hit_lower_mask = current_future_prices[active_indices_after_upper] <= lower_barrier[active_indices_after_upper]
        hit_lower_indices = active_indices_after_upper[hit_lower_mask]
        if len(hit_lower_indices) > 0:
            labels[hit_lower_indices] = [0, 1, 0] # Sell

    n_buy = np.sum(labels[:, 0])
    n_sell = np.sum(labels[:, 1])
    n_hold = np.sum(labels[:, 2])
    logger.info(f"Labels générés: Buy={int(n_buy)}, Sell={int(n_sell)}, Hold={int(n_hold)}")
    if n_buy == 0 and n_sell == 0 and n_hold == n_samples:
         logger.warning("Aucun label Buy/Sell généré. Vérifiez la logique ou les données.")
    return labels

labels = _generate_labels(sequences, data) # Utiliser le dataframe normalisé 'data'

# 6. Split data
def _split_data(sequences: np.ndarray, labels: np.ndarray, test_size=0.2):
    split_idx = int(len(sequences) * (1 - test_size))
    train_seq, test_seq = sequences[:split_idx], sequences[split_idx:]
    train_labels, test_labels = labels[:split_idx], labels[split_idx:]
    logger.info(f"Split data: Train ({train_seq.shape}, {train_labels.shape}), Test ({test_seq.shape}, {test_labels.shape})")
    return (train_seq, train_labels), (test_seq, test_labels)

(train_seq, train_labels), (val_seq, val_labels) = _split_data(sequences, labels)

# Créer les tuples finaux pour l'entraînement
train_data = (train_seq, train_labels)
val_data = (val_seq, val_labels)
logger.info("Préparation des données terminée.")


## 5. Initialisation de l'Expérience W&B

In [ ]:
import wandb

# Initialisation de l'expérience W&B pour le suivi
try:
    # Utiliser les paramètres définis plus haut dans la config W&B
    wandb_config = {
        'exchange': exchange_name,
        'pair': pair,
        'timeframe': timeframe,
        'start_date': start_date,
        'end_date': end_date,
        'learning_rate': config.learning_rate,
        'batch_size': config.batch_size,
        'epochs': config.epochs,
        'time_window': time_window,
        'cnn_filters': config.cnn_filters,
        'lstm_units': config.lstm_units,
        'dense_units': config.dense_units
    }
    wandb.init(project="morningstar-colab-training-dl", entity="cabrelkaka-morningstar", config=wandb_config)
    logger.info("Wandb initialisé avec succès.")
except Exception as e:
    logger.error(f"Erreur lors de l'initialisation de Wandb: {e}")
    wandb_active = False
else:
    wandb_active = True

## 6. Chargement et Compilation du Modèle

In [ ]:
from Morningstar.model.architecture.morningstar_model import MorningstarTradingModel

# Déterminer input_shape et num_classes à partir des données préparées
input_shape = train_data[0].shape[1:] # (time_window, num_features)
num_classes = train_data[1].shape[1] # Nombre de classes (Buy, Sell, Hold -> 3)

logger.info(f"Initialisation du modèle avec input_shape={input_shape}, num_classes={num_classes}")

# Instancier le modèle
model_instance = MorningstarTradingModel(
    input_shape=input_shape,
    num_classes=num_classes,
    cnn_filters=config.cnn_filters,
    lstm_units=config.lstm_units,
    dense_units=config.dense_units
)

# Compiler le modèle
model_instance.compile_model(learning_rate=config.learning_rate)
logger.info("Modèle compilé.")
model_instance.model.summary() # Afficher le résumé du modèle

## 7. Configuration des Callbacks

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Définition des callbacks pour l'entraînement
callbacks = [
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', verbose=1),
    EarlyStopping(patience=15, monitor='val_loss', restore_best_weights=True, verbose=1)
]

if wandb_active:
    try:
        from wandb.keras import WandbCallback
        callbacks.append(WandbCallback(save_model=False)) # Ne pas sauvegarder le modèle via W&B, on le fait déjà
        logger.info("Callback Wandb ajouté.")
    except ImportError:
        logger.warning("Impossible d'importer WandbCallback. Le suivi W&B sera limité.")

logger.info(f"Callbacks configurés: {callbacks}")

## 8. Entraînement du Modèle

L'entraînement peut prendre du temps en fonction de la taille des données et du nombre d'époques.

In [ ]:
logger.info("Début de l'entraînement...")

strategy = tf.distribute.MirroredStrategy() if len(gpu_devices) > 1 else tf.distribute.get_strategy()
logger.info(f"Utilisation de la stratégie de distribution: {strategy.__class__.__name__}")

with strategy.scope():
    # Re-créer et compiler le modèle dans le scope est recommandé pour MirroredStrategy
    model_instance_scoped = MorningstarTradingModel(
        input_shape=input_shape,
        num_classes=num_classes,
        cnn_filters=config.cnn_filters,
        lstm_units=config.lstm_units,
        dense_units=config.dense_units
    )
    model_instance_scoped.compile_model(learning_rate=config.learning_rate)
    logger.info("Modèle recréé et compilé dans le scope de la stratégie.")

    history = model_instance_scoped.model.fit(
        x=train_data[0], 
        y=train_data[1], 
        epochs=config.epochs,
        batch_size=config.batch_size,
        validation_data=val_data, 
        callbacks=callbacks
    )

logger.info("Entraînement terminé.")

final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
logger.info(f"Perte finale (entraînement): {final_train_loss:.4f}")
logger.info(f"Perte finale (validation): {final_val_loss:.4f}")

if wandb_active:
    wandb.finish()

## 9. Sauvegarde sur Google Drive (Optionnel)

In [ ]:
from google.colab import drive
import os

try:
    drive.mount('/content/drive', force_remount=True)
    drive_path = '/content/drive/MyDrive/Morningstar_Models/' # Chemin sur votre Drive
    os.makedirs(drive_path, exist_ok=True) 
    model_save_path = os.path.join(drive_path, f'best_model_{pair.replace("/", "_")}_{timeframe}_{end_date}.h5')
    
    if os.path.exists('best_model.h5'):
        !cp best_model.h5 "{model_save_path}"
        logger.info(f"Modèle sauvegardé sur Google Drive: {model_save_path}")
    else:
        logger.warning("Le fichier 'best_model.h5' n'a pas été trouvé. Sauvegarde sur Drive annulée.")

except Exception as e:
    logger.error(f"Erreur lors de la sauvegarde sur Google Drive: {e}")

## 10. Téléchargement du Modèle Entraîné

Exécutez la cellule suivante pour télécharger le fichier `best_model.h5` sur votre machine locale.

In [ ]:
from google.colab import files
import os

if os.path.exists('best_model.h5'):
    print("Téléchargement de best_model.h5...")
    files.download('best_model.h5')
else:
    logger.error("Le fichier 'best_model.h5' n'a pas été trouvé pour le téléchargement.")